In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Essential_Thrombocythemia"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Essential_Thrombocythemia/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Essential_Thrombocythemia/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Essential_Thrombocythemia/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Essential_Thrombocythemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os

# Step 1: Look for directories related to Adrenocortical Cancer
tcga_subdirs = os.listdir(tcga_root_dir)
print(f"Available TCGA subdirectories: {tcga_subdirs}")

# Look for directory related to Adrenocortical Cancer
target_dir = None
for subdir in tcga_subdirs:
    # Look for exact match or synonymous terms
    if trait.lower() in subdir.lower() or "ACC" in subdir:
        target_dir = subdir
        break

if target_dir is None:
    print(f"No suitable directory found for {trait}.")
    # Mark the task as completed by creating a JSON record indicating data is not available
    validate_and_save_cohort_info(is_final=False, cohort="TCGA", info_path=json_path, 
                                 is_gene_available=False, is_trait_available=False)
    exit()  # Exit the program

# Step 2: Get file paths for the selected directory
cohort_dir = os.path.join(tcga_root_dir, target_dir)
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

print(f"Selected directory: {target_dir}")
print(f"Clinical data file: {clinical_file_path}")
print(f"Genetic data file: {genetic_file_path}")

# Step 3: Load clinical and genetic data
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')

# Step 4: Print column names of clinical data
print("\nClinical data columns:")
print(clinical_df.columns.tolist())

# Additional basic information
print(f"\nClinical data shape: {clinical_df.shape}")
print(f"Genetic data shape: {genetic_df.shape}")


Available TCGA subdirectories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Canc


Clinical data columns:
['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'atypical_mitotic_figures', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_M', 'ct_scan_findings', 'cytoplasm_presence_less_than_equal_25_percent', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'diffuse_architecture', 'distant_metastasis_anatomic_site', 'excess_adrenal_hormone_diagnosis_method_type', 'excess_adrenal_hormone_history_type', 'form_completion_date', 'gender', 'germline_testing_performed', 'histologic_disease_progression_present_indicator', 'histological_type', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histology', 'icd_o_3_site', 'informed_consen

### Step 2: Find Candidate Demographic Features

In [3]:
# Identify candidate age and gender columns
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

# Load the clinical data file path
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(os.path.join(tcga_root_dir, 'TCGA_Adrenocortical_Cancer_(ACC)'))

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
if candidate_age_cols:
    age_preview = {col: clinical_df[col].head(5).tolist() for col in candidate_age_cols}
    print("Age columns preview:")
    print(age_preview)

# Extract and preview gender columns
if candidate_gender_cols:
    gender_preview = {col: clinical_df[col].head(5).tolist() for col in candidate_gender_cols}
    print("Gender columns preview:")
    print(gender_preview)


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [58, 44, 23, 23, 30], 'days_to_birth': [-21496, -16090, -8624, -8451, -11171]}
Gender columns preview:
{'gender': ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'MALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine potential age columns
print("Evaluating age columns:")
for col_name, values in {'age_at_initial_pathologic_diagnosis': [58, 44, 23, 23, 30], 'days_to_birth': [-21496, -16090, -8624, -8451, -11171]}.items():
    missing_count = sum(1 for v in values if v is None)
    missing_percentage = missing_count / len(values) * 100 if values else 0
    print(f"  {col_name}: {values}, Missing: {missing_percentage}%")

# Examine potential gender columns
print("Evaluating gender columns:")
for col_name, values in {'gender': ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'MALE']}.items():
    missing_count = sum(1 for v in values if v is None)
    missing_percentage = missing_count / len(values) * 100 if values else 0
    print(f"  {col_name}: {values}, Missing: {missing_percentage}%")

# Select the best columns for age and gender
age_col = "age_at_initial_pathologic_diagnosis"  # Contains direct age values which are easier to interpret
gender_col = "gender"  # Contains standard gender information

# Print the chosen columns
print(f"\nChosen age column: {age_col}")
print(f"Chosen gender column: {gender_col}")


Evaluating age columns:
  age_at_initial_pathologic_diagnosis: [58, 44, 23, 23, 30], Missing: 0.0%
  days_to_birth: [-21496, -16090, -8624, -8451, -11171], Missing: 0.0%
Evaluating gender columns:
  gender: ['MALE', 'FEMALE', 'FEMALE', 'FEMALE', 'MALE'], Missing: 0.0%

Chosen age column: age_at_initial_pathologic_diagnosis
Chosen gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize the clinical features
# Get file paths
cohort_dir = os.path.join(tcga_root_dir, 'TCGA_Adrenocortical_Cancer_(ACC)')
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Create standardized clinical features dataframe with trait, age, and gender
# The trait for Adrenocortical Cancer is based on tumor/normal classification
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using predefined trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print(f"Clinical data shape: {clinical_features.shape}")
print(clinical_features.head())

# Step 2: Normalize gene symbols in gene expression data
# Transpose the genetic data to have genes as rows
genetic_data = genetic_df.copy()

# Normalize gene symbols using the NCBI Gene database synonyms
normalized_gene_data = normalize_gene_symbols_in_index(genetic_data)

# Save normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")
print(f"Normalized gene data shape: {normalized_gene_data.shape}")

# Step 3: Link clinical and genetic data
# Transpose genetic data to get samples as rows, genes as columns
genetic_data_transposed = normalized_gene_data.T

# Ensure clinical and genetic data have the same samples (index values)
common_samples = clinical_features.index.intersection(genetic_data_transposed.index)
clinical_subset = clinical_features.loc[common_samples]
genetic_subset = genetic_data_transposed.loc[common_samples]

# Combine clinical and genetic data
linked_data = pd.concat([clinical_subset, genetic_subset], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# Step 4: Handle missing values
linked_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values - linked data shape: {linked_data.shape}")

# Step 5: Determine biased features
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait=trait)
print(f"After removing biased features - linked data shape: {linked_data.shape}")

# Step 6: Validate data quality and save cohort info
# First check if we have both gene and trait data
is_gene_available = linked_data.shape[1] > 3  # More than just trait, Age, Gender
is_trait_available = trait in linked_data.columns

# Take notes of special findings
notes = f"TCGA Adrenocortical Cancer dataset processed. Used tumor/normal classification as the trait."

# Validate the data quality
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=notes
)

# Step 7: Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality concerns")

Clinical data saved to ../../output/preprocess/Essential_Thrombocythemia/clinical_data/TCGA.csv
Clinical data shape: (92, 3)
                 Essential_Thrombocythemia  Age  Gender
sampleID                                               
TCGA-OR-A5J1-01                          1   58       1
TCGA-OR-A5J2-01                          1   44       0
TCGA-OR-A5J3-01                          1   23       0
TCGA-OR-A5J4-01                          1   23       0
TCGA-OR-A5J5-01                          1   30       1


Normalized gene data saved to ../../output/preprocess/Essential_Thrombocythemia/gene_data/TCGA.csv
Normalized gene data shape: (19848, 79)
Linked data shape: (79, 19851)


After handling missing values - linked data shape: (79, 19851)
Quartiles for 'Essential_Thrombocythemia':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Essential_Thrombocythemia' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 35.0
  50% (Median): 49.0
  75%: 59.5
Min: 14
Max: 77
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 31 occurrences. This represents 39.24% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

After removing biased features - linked data shape: (79, 19851)
Linked data not saved due to quality concerns
